In [1]:
import numpy as np
import pandas as pd
import keras
import warnings
warnings.filterwarnings(action='ignore')
import sys

from keras.models import Sequential
from keras.layers import Dense,Dropout

### Data 불러오기

In [2]:
data = pd.read_csv('data_preprocessing.csv')
data

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,1,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,1,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,1,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,1,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,1,60,19,20,1433500
...,...,...,...,...,...,...,...,...,...,...,...
2691680,2020,3,충북,충주시,휴양콘도 운영업,1,1,30,3,4,43300
2691681,2020,3,충북,충주시,휴양콘도 운영업,1,1,40,3,3,35000
2691682,2020,3,충북,충주시,휴양콘도 운영업,1,1,50,4,6,188000
2691683,2020,3,충북,충주시,휴양콘도 운영업,1,1,60,3,3,194000


### Train, Test 구분

In [3]:
Train_data = data.loc[(data['year']==2020)&((data['month']==1)|(data['month']==2)|
                                              (data['month']==3))]
Train_data_X = Train_data.drop(['AMT','year','CNT','CSTMR_CNT'],axis=1)
Train_data = Train_data.drop(['CNT','year','CSTMR_CNT'],axis=1)
print('Train_data :',Train_data.shape)

Train_data : (521750, 8)


### SET Pred_Dataframe

In [4]:
def Pred_data_def(raw_data):
    temp         = []
    CARD_CCG_NM  = raw_data['CARD_CCG_NM'].unique()
    SEX          = raw_data['SEX'].unique()
    Region_diff  = raw_data['region_diff'].unique()
    AGE          = raw_data['AGE'].unique()
    month        = [4,7]

    for ccg in CARD_CCG_NM:
        for age in AGE: 
            for sex in SEX:
                for region in Region_diff:
                    for i in month:
                        temp.append([i, ccg, sex, region, age])

    temp = np.array(temp)
    Pred_data = pd.DataFrame(data= temp, columns= ['month', 'CARD_CCG_NM', 'SEX',
                                                  'region_diff', 'AGE'])
    return Pred_data

### Model

In [30]:
### 세팅중

final_data  = []
sido_list = Train_data['CARD_SIDO_NM'].unique()[3:4]
std_list  = Train_data['STD_CLSS_NM'].unique()[29:33]
dum_col   = ['CARD_CCG_NM','SEX','region_diff','AGE']

for i, sido in enumerate(sido_list):
    for j, std in enumerate(std_list): 
        Pred_data = Pred_data_def(Train_data_X)     
        
        Train_data_en = pd.get_dummies(Train_data, columns=dum_col)
        Pred_data_en = pd.get_dummies(Pred_data, columns=dum_col)        
        
        df_train1 = Train_data_en[(Train_data['CARD_SIDO_NM'] == sido) & (Train_data_en['STD_CLSS_NM'] == std)]                
        
        print("★ ", i, j)
        
        #DF가 비어있을때 학습X
        if( df_train1.empty): 
            print("train가 비어있음")
            continue

        y_train = df_train1['AMT'].to_numpy(dtype=np.float32).reshape(-1,1)
        X_train = df_train1.drop(['AMT','CARD_SIDO_NM','STD_CLSS_NM'],axis=1).to_numpy(dtype=np.float32)
        X_test = Pred_data_en.to_numpy(dtype=np.float32)        

        print("sido: {}번째 / std: {}번째".format(sido, std))
        print('TRAIN : ',X_train.shape,y_train.shape,
             '\nTEST : ',X_test.shape)

        print('-'*30)

        num_hidden_units = 150
        input_dim = X_train.shape[1]

        model = Sequential()
        model.add(Dense(num_hidden_units,input_dim = input_dim, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units, kernel_initializer='normal',activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dense(1))
        model.compile(loss='MeanAbsolutePercentageError', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=1000, batch_size=3, verbose=1)
        
        # 예측
        pred = model.predict(X_test)
        Pred_data['AMT'] = np.round(pred, 0)
        Pred_data['REG_YYMM'] = ('20200' + Pred_data['month']).astype('int64')
        Pred_data['CARD_SIDO_NM'] = sido
        Pred_data['STD_CLSS_NM']  = std
        Pred_data = Pred_data[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
        Pred_data = Pred_data.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False) 
        fin = pd.DataFrame(Pred_data[0:1])
        final_data.append(fin.values)
        fin = pd.DataFrame(Pred_data[1:2])
        final_data.append(fin.values)

★  0 0
sido: 경북번째 / std: 한식 음식점업번째
TRAIN :  (2573, 240) (2573, 1) 
TEST :  (19068, 240)
------------------------------
Epoch 1/1000
858/858 [==============================] - 1s 2ms/step - loss: 96.4085 - accuracy: 0.0000e+00
Epoch 2/1000
858/858 [==============================] - 1s 2ms/step - loss: 92.9513 - accuracy: 0.0000e+00
Epoch 3/1000
858/858 [==============================] - 1s 2ms/step - loss: 91.2429 - accuracy: 0.0000e+00
Epoch 4/1000
858/858 [==============================] - 1s 2ms/step - loss: 90.1308 - accuracy: 0.0000e+00
Epoch 5/1000
858/858 [==============================] - 1s 2ms/step - loss: 88.2810 - accuracy: 0.0000e+00
Epoch 6/1000
858/858 [==============================] - 1s 2ms/step - loss: 85.6464 - accuracy: 0.0000e+00
Epoch 7/1000
858/858 [==============================] - 1s 2ms/step - loss: 78.9173 - accuracy: 0.0000e+00
Epoch 8/1000
858/858 [==============================] - 1s 2ms/step - loss: 69.8760 - accuracy: 0.0000e+00
Epoch 9/1000
858/858 [===

858/858 [==============================] - 1s 1ms/step - loss: 24.6299 - accuracy: 0.0000e+00
Epoch 76/1000
858/858 [==============================] - 1s 1ms/step - loss: 24.5525 - accuracy: 0.0000e+00
Epoch 77/1000
858/858 [==============================] - 1s 1ms/step - loss: 24.3143 - accuracy: 0.0000e+00
Epoch 78/1000
858/858 [==============================] - 1s 1ms/step - loss: 24.1934 - accuracy: 0.0000e+00
Epoch 79/1000
858/858 [==============================] - 1s 1ms/step - loss: 23.9259 - accuracy: 0.0000e+00
Epoch 80/1000
858/858 [==============================] - 1s 2ms/step - loss: 23.9489 - accuracy: 0.0000e+00A: 0s - loss: 24.2678 - accuracy: 
Epoch 81/1000
858/858 [==============================] - 1s 1ms/step - loss: 23.6687 - accuracy: 0.0000e+00
Epoch 82/1000
858/858 [==============================] - 1s 1ms/step - loss: 23.6709 - accuracy: 0.0000e+00
Epoch 83/1000
858/858 [==============================] - 1s 1ms/step - loss: 24.2878 - accuracy: 0.0000e+00
Epoch 84

858/858 [==============================] - 1s 2ms/step - loss: 21.7491 - accuracy: 0.0000e+00
Epoch 149/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.4568 - accuracy: 0.0000e+00
Epoch 150/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.6739 - accuracy: 0.0000e+00
Epoch 151/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.3216 - accuracy: 0.0000e+00
Epoch 152/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.7755 - accuracy: 0.0000e+00
Epoch 153/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.8246 - accuracy: 0.0000e+00
Epoch 154/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.0226 - accuracy: 0.0000e+00
Epoch 155/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.4422 - accuracy: 0.0000e+00 1s - loss: 17.669
Epoch 156/1000
858/858 [==============================] - 1s 1ms/step - loss: 21.6940 - accuracy: 0.0000e+00
Epoch 157/1000
8

858/858 [==============================] - 1s 1ms/step - loss: 20.1824 - accuracy: 0.0000e+00
Epoch 223/1000
858/858 [==============================] - 1s 1ms/step - loss: 20.7084 - accuracy: 0.0000e+00
Epoch 224/1000
858/858 [==============================] - 1s 1ms/step - loss: 20.1143 - accuracy: 0.0000e+00
Epoch 225/1000
858/858 [==============================] - 1s 1ms/step - loss: 20.6175 - accuracy: 0.0000e+00
Epoch 226/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.6077 - accuracy: 0.0000e+00
Epoch 227/1000
858/858 [==============================] - 1s 1ms/step - loss: 20.0233 - accuracy: 0.0000e+00
Epoch 228/1000
858/858 [==============================] - 1s 1ms/step - loss: 20.1904 - accuracy: 0.0000e+00
Epoch 229/1000
858/858 [==============================] - 1s 1ms/step - loss: 20.3035 - accuracy: 0.0000e+00
Epoch 230/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.7830 - accuracy: 0.0000e+00
Epoch 231/1000
858/858 [==========

858/858 [==============================] - 1s 1ms/step - loss: 19.3669 - accuracy: 0.0000e+00
Epoch 298/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.3396 - accuracy: 0.0000e+00
Epoch 299/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.3000 - accuracy: 0.0000e+00
Epoch 300/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.4400 - accuracy: 0.0000e+00
Epoch 301/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.4382 - accuracy: 0.0000e+00
Epoch 302/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.4809 - accuracy: 0.0000e+00
Epoch 303/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.4923 - accuracy: 0.0000e+00
Epoch 304/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.6539 - accuracy: 0.0000e+00
Epoch 305/1000
858/858 [==============================] - 1s 1ms/step - loss: 19.1098 - accuracy: 0.0000e+00
Epoch 306/1000
858/858 [==========

...

57/57 [==============================] - 0s 1ms/step - loss: 20.8260 - accuracy: 0.0000e+00
Epoch 661/1000
57/57 [==============================] - 0s 1ms/step - loss: 22.3230 - accuracy: 0.0000e+00
Epoch 662/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.4865 - accuracy: 0.0000e+00
Epoch 663/1000
57/57 [==============================] - 0s 1ms/step - loss: 24.7057 - accuracy: 0.0000e+00
Epoch 664/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.7376 - accuracy: 0.0000e+00
Epoch 665/1000
57/57 [==============================] - 0s 1ms/step - loss: 22.3346 - accuracy: 0.0000e+00
Epoch 666/1000
57/57 [==============================] - 0s 1ms/step - loss: 22.9308 - accuracy: 0.0000e+00
Epoch 667/1000
57/57 [==============================] - 0s 1ms/step - loss: 23.0145 - accuracy: 0.0000e+00
Epoch 668/1000
57/57 [==============================] - 0s 1ms/step - loss: 22.5098 - accuracy: 0.0000e+00
Epoch 669/1000
57/57 [==============================

57/57 [==============================] - 0s 1ms/step - loss: 21.4579 - accuracy: 0.0000e+00
Epoch 737/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.1805 - accuracy: 0.0000e+00
Epoch 738/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.2355 - accuracy: 0.0000e+00
Epoch 739/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.4512 - accuracy: 0.0000e+00
Epoch 740/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.9695 - accuracy: 0.0000e+00
Epoch 741/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.4743 - accuracy: 0.0000e+00
Epoch 742/1000
57/57 [==============================] - 0s 1ms/step - loss: 23.3168 - accuracy: 0.0000e+00
Epoch 743/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.0425 - accuracy: 0.0000e+00
Epoch 744/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.3119 - accuracy: 0.0000e+00
Epoch 745/1000
57/57 [==============================

57/57 [==============================] - 0s 1ms/step - loss: 21.2315 - accuracy: 0.0000e+00
Epoch 813/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.6575 - accuracy: 0.0000e+00
Epoch 814/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.9899 - accuracy: 0.0000e+00
Epoch 815/1000
57/57 [==============================] - 0s 1ms/step - loss: 21.8706 - accuracy: 0.0000e+00
Epoch 816/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.8761 - accuracy: 0.0000e+00
Epoch 817/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.2050 - accuracy: 0.0000e+00
Epoch 818/1000
57/57 [==============================] - 0s 1ms/step - loss: 24.8180 - accuracy: 0.0000e+00
Epoch 819/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.7028 - accuracy: 0.0000e+00
Epoch 820/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.3016 - accuracy: 0.0000e+00
Epoch 821/1000
57/57 [==============================

57/57 [==============================] - 0s 1ms/step - loss: 22.7137 - accuracy: 0.0000e+00
Epoch 889/1000
57/57 [==============================] - 0s 1ms/step - loss: 18.1034 - accuracy: 0.0000e+00
Epoch 890/1000
57/57 [==============================] - 0s 1ms/step - loss: 19.7798 - accuracy: 0.0000e+00
Epoch 891/1000
57/57 [==============================] - 0s 1ms/step - loss: 19.7888 - accuracy: 0.0000e+00
Epoch 892/1000
57/57 [==============================] - 0s 1ms/step - loss: 19.6713 - accuracy: 0.0000e+00
Epoch 893/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.2189 - accuracy: 0.0000e+00
Epoch 894/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.4725 - accuracy: 0.0000e+00
Epoch 895/1000
57/57 [==============================] - 0s 1ms/step - loss: 18.4275 - accuracy: 0.0000e+00
Epoch 896/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.6159 - accuracy: 0.0000e+00
Epoch 897/1000
57/57 [==============================

57/57 [==============================] - 0s 1ms/step - loss: 19.5347 - accuracy: 0.0000e+00
Epoch 965/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.4399 - accuracy: 0.0000e+00
Epoch 966/1000
57/57 [==============================] - 0s 1ms/step - loss: 18.4048 - accuracy: 0.0000e+00
Epoch 967/1000
57/57 [==============================] - 0s 1ms/step - loss: 19.1498 - accuracy: 0.0000e+00
Epoch 968/1000
57/57 [==============================] - 0s 1ms/step - loss: 18.3806 - accuracy: 0.0000e+00
Epoch 969/1000
57/57 [==============================] - 0s 1ms/step - loss: 19.7384 - accuracy: 0.0000e+00
Epoch 970/1000
57/57 [==============================] - 0s 1ms/step - loss: 17.7994 - accuracy: 0.0000e+00
Epoch 971/1000
57/57 [==============================] - 0s 1ms/step - loss: 20.0096 - accuracy: 0.0000e+00
Epoch 972/1000
57/57 [==============================] - 0s 1ms/step - loss: 19.8524 - accuracy: 0.0000e+00
Epoch 973/1000
57/57 [==============================

### Pred_DF with AMT

In [31]:
col = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']
Final_data = pd.DataFrame(columns=col)
for data in range(len(final_data)):
    frist_df = pd.DataFrame(final_data[data],columns=col)
    Final_data = Final_data.append(frist_df)
Final_data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,경북,한식 음식점업,1.75037e+10
0,202007,경북,한식 음식점업,2.92645e+10
0,202004,경북,호텔업,1.25239e+09
0,202007,경북,호텔업,2.72577e+09
0,202004,경북,화장품 및 방향제 소매업,1.75261e+09
0,202007,경북,화장품 및 방향제 소매업,3.51395e+09
0,202004,경북,휴양콘도 운영업,1.96342e+09
0,202007,경북,휴양콘도 운영업,4.06483e+09


### Save to csv

In [32]:
submission = pd.read_csv('submission_mingsu.csv',index_col = 0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(Final_data, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission_mingsu.csv', encoding='utf-8-sig')
submission.head(10)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,NaN
1,202004,강원,골프장 운영업,NaN
2,202004,강원,과실 및 채소 소매업,NaN
3,202004,강원,관광 민예품 및 선물용품 소매업,NaN
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
5,202004,강원,그외 기타 스포츠시설 운영업,NaN
6,202004,강원,그외 기타 종합 소매업,NaN
7,202004,강원,기타 대형 종합 소매업,NaN
8,202004,강원,기타 수상오락 서비스업,NaN
